In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests

driver_path = "./msedgedriver.exe"

service = EdgeService(executable_path=driver_path)
driver = webdriver.Edge(service=service)

driver.get("https://www.hyundai.com/kr/ko/e/customer/branch?brCd=D46")

In [1]:
import requests

https://www.hyundai.com/kr/ko/gw/customer-support/v1/purchase-guide/agencies?latitude=37.4646651&longitude=127.0430687&mapLatitude=&mapLongitude=&radius=&pageSize=10&pageNo=1&searchFilter=&agencyTypeCode=&deliveryAreaCode=01&localAreaCode=0101&findType=W

느낌에 localAreaCode로 잡을 수 있을 것 같다.


In [10]:
payload= {
"latitude":"37.4646651",
"longitude":"127.0430687",
"mapLatitude":"",
"mapLongitude":"",
"radius":"",
"pageSize":"10",
"pageNo":"1",
"searchFilter":"",
"agencyTypeCode":"",
"deliveryAreaCode":"01",
"localAreaCode":"0101",
"findType":"W",
}

In [14]:
import requests

url = "https://www.hyundai.com/kr/ko/gw/customer-support/v1/purchase-guide/agencies"

params = {
    "latitude": 37.4646651,
    "longitude": 127.0430687,
    "mapLatitude": "",
    "mapLongitude": "",
    "radius": "",
    "pageSize": 10,
    "pageNo": 1,
    "searchFilter": "",
    "agencyTypeCode": "",
    "deliveryAreaCode": "01",
    "localAreaCode": "0101",
    "findType": "W",
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://www.hyundai.com/kr/ko/e/vehicles/purchase-guide",
    "Origin": "https://www.hyundai.com",
}

r = requests.get(url, params=params, headers=headers, timeout=20)
print(r.status_code)



200


In [20]:
import json
import pandas as pd

r = r.text.strip("'")


# 1) JSON 파싱
obj = json.loads(r)
stores = obj["data"]["list"]

In [22]:


# 2) "유의미한" 컬럼만 추출
keep_cols = [
    "agencyCode", "agencyName", "agencyTypeCode",
    "agencyAddress", "agencyTel",
    "lattitude", "longitude",
    "distance", "displayCarCount", "carMasterCount"
]

rows = []
for s in stores:
    row = {k: s.get(k) for k in keep_cols}

    # 3) 타입 정리(숫자 변환)
    row["distance_km"] = float(row["distance"]) if row.get("distance") not in (None, "") else None
    row["lat"] = float(row["lattitude"]) if row.get("lattitude") not in (None, "") else None
    row["lon"] = float(row["longitude"]) if row.get("longitude") not in (None, "") else None
    row["displayCarCount"] = int(row["displayCarCount"]) if row.get("displayCarCount") not in (None, "") else None
    row["carMasterCount"] = int(row["carMasterCount"]) if row.get("carMasterCount") not in (None, "") else None

    # 4) 전시장 여부(대충 displayCarCount > 0이면 전시장 성격)
    row["is_showroom"] = (row["displayCarCount"] or 0) > 0

    # 5) 타입코드 라벨링(네 데이터 기준: 1=지점/전시장, 2=대리점 느낌)
    type_map = {"1": "지점/전시장", "2": "대리점"}
    row["agencyType"] = type_map.get(str(row["agencyTypeCode"]), "기타")

    rows.append(row)

df = pd.DataFrame(rows)

# 6) 보기 좋게 정렬: 가까운 순
df = df.sort_values(["distance_km", "agencyName"], ascending=[True, True]).reset_index(drop=True)

# 7) 최종 보여줄 컬럼만 선택(원하면 더 추가 가능)
df_final = df[
    ["agencyName", "agencyType", "distance_km", "agencyAddress", "agencyTel", "is_showroom", "displayCarCount", "carMasterCount", "lat", "lon", "agencyCode"]
]

df

,agencyCode,agencyName,agencyTypeCode,agencyAddress,agencyTel,lattitude,longitude,distance,displayCarCount,carMasterCount,distance_km,lat,lon,is_showroom,agencyType
0,DB39004,도곡중앙,2,서울특별시 강남구 언주로 122 (도곡동) (06295),02-3463-5200,37.487082477134,127.051017587398,2.6,0,18,2.6,37.487082,127.051018,False,대리점
1,B39,강남대로,1,서울특별시 강남구 강남대로 292 현대자동차(주) 강남대로지점 전시장 (06258),02-3461-1990,37.4893966742,127.032205066703,2.9,8,36,2.9,37.489397,127.032205,True,지점/전시장
2,DB44007,언주,2,서울특별시 강남구 도곡로 251 1층 전시장 2층 사무실 (06230),02-6203-8200,37.494250167536,127.045747575968,3.3,0,15,3.3,37.494250,127.045748,False,대리점
3,DB44006,신대치,2,서울특별시 강남구 삼성로 324 (대치동) (06191),02-556-8777,37.500837553551,127.060137997529,4.3,0,18,4.3,37.500838,127.060138,False,대리점
4,X83,대치,1,"서울특별시 강남구 영동대로 302 (구상빌딩1,2층) (06177)",02-564-0365,37.501514402321,127.067769302362,4.6,3,24,4.6,37.501514,127.067769,True,지점/전시장
5,DB44003,남서울,2,서울특별시 강남구 선릉로 531 (역삼동) (06149),02-552-4458,37.506759809216,127.046935065841,4.7,1,19,4.7,37.506760,127.046935,True,대리점
6,DB47004,신사,2,"서울특별시 강남구 봉은사로 115 1층 (논현동, 노벨테크빌딩) (06120)",02-515-8111,37.50527498984,127.02613029035,4.8,0,12,4.8,37.505275,127.026130,False,대리점
7,DX83004,율곡,2,서울특별시 강남구 밤고개로 252 (율현동) (06377),02-445-0369,37.471787697276,127.106821695827,5.7,2,15,5.7,37.471788,127.106822,True,대리점
8,DX83003,수서역,2,서울특별시 강남구 광평로 280 1층 107호 (수서동) (06367),02-3412-0558,37.487319163149,127.103108072457,5.9,0,18,5.9,37.487319,127.103108,False,대리점
9,B44,강남청담,1,"서울특별시 강남구 삼성로 648 1층, 현대자동차(주) (06084)",02-558-2811,37.518007780896,127.051024607576,6,2,18,6.0,37.518008,127.051025,True,지점/전시장


In [32]:
r = requests.get("https://www.hyundai.com/kr/ko/gw/common/v1/common/common-code/E/Z028", params=params, headers=headers, timeout=20)
print(r.status_code)


200


In [33]:
r.text

'{"data":[{"systemTypeCode":"E","codeTypeCode":"Z028","code":"01","codeName":"서울특별시","codeAbbrName":"서울","chrrName1":"B","chrrName2":"1100000000","chrrName3":"001","snNo":1,"repnCarInfos":[]},{"systemTypeCode":"E","codeTypeCode":"Z028","code":"06","codeName":"인천광역시","codeAbbrName":"인천","chrrName1":"D","chrrName2":"2800000000","chrrName3":"006","snNo":2,"repnCarInfos":[]},{"systemTypeCode":"E","codeTypeCode":"Z028","code":"07","codeName":"경기도","codeAbbrName":"경기","chrrName1":"E","chrrName2":"4100000000","chrrName3":"007","snNo":3,"repnCarInfos":[]},{"systemTypeCode":"E","codeTypeCode":"Z028","code":"02","codeName":"강원특별자치도","codeAbbrName":"강원","chrrName1":"F","chrrName2":"4200000000","chrrName3":"002","snNo":4,"repnCarInfos":[]},{"systemTypeCode":"E","codeTypeCode":"Z028","code":"05","codeName":"충청남도","codeAbbrName":"충남","chrrName1":"I","chrrName2":"4400000000","chrrName3":"005","snNo":6,"repnCarInfos":[]},{"systemTypeCode":"E","codeTypeCode":"Z028","code":"03","codeName":"대전광역시","codeA

In [34]:
import json
import pandas as pd

r_sido = r.text.strip("'")


# 1) JSON 파싱
obj = json.loads(r_sido)
sido= obj["data"]

In [36]:
sido_df=pd.DataFrame(obj['data'])

In [46]:
params = {
    "latitude": 37.4646651,
    "longitude": 127.0430687,
    "mapLatitude": "",
    "mapLongitude": "",
    "radius": "",
    "pageSize": 10,
    "pageNo": 1,
    "searchFilter": "",
    "agencyTypeCode": "",
    "deliveryAreaCode": "14",
    "localAreaCode": "0101",
    "findType": "W",
}

In [58]:
sido_df.head() #params에서 deliveryAreaCode 돌기

,systemTypeCode,codeTypeCode,code,codeName,codeAbbrName,chrrName1,chrrName2,chrrName3,snNo,repnCarInfos
0,E,Z028,01,서울특별시,서울,B,1100000000,001,1,[]
1,E,Z028,06,인천광역시,인천,D,2800000000,006,2,[]
2,E,Z028,07,경기도,경기,E,4100000000,007,3,[]
3,E,Z028,02,강원특별자치도,강원,F,4200000000,002,4,[]
4,E,Z028,05,충청남도,충남,I,4400000000,005,6,[]


In [59]:
sido_list=list(sido_df['code'])

In [70]:
sido_list.__len__()

16

In [47]:
r = requests.get("https://www.hyundai.com/kr/ko/gw/common/v1/common/common-code/E/Z029", params=params, headers=headers, timeout=20)
print(r.status_code)


import json
import pandas as pd

r_gu = r.text.strip("'")


# 1) JSON 파싱
obj = json.loads(r_gu)
gugun= obj["data"]
gugun_df=pd.DataFrame(gugun)

200


In [48]:
gugun_df.head()

,systemTypeCode,codeTypeCode,code,codeName,chrrName1,chrrName2,chrrName3,repnCarInfos
0,E,Z029,0101,강남구,B0,1168000000,135,[]
1,E,Z029,1109,사상구,P0,2653000000,617,[]
2,E,Z029,1108,북구,P0,2632000000,616,[]
3,E,Z029,1107,부산진구,P0,2623000000,614,[]
4,E,Z029,1106,동래구,P0,2626000000,607,[]


In [50]:
gugun_list=list(gugun_df['code'])

In [51]:
gugun_list

['0101',
 '1109',
 '1108',
 '1107',
 '1106',
 '1105',
 '1104',
 '1103',
 '1102',
 '1101',
 '1022',
 '1021',
 '1020',
 '1019',
 '1018',
 '1017',
 '1110',
 '1111',
 '1305',
 '1304',
 '1303',
 '1302',
 '1301',
 '1205',
 '1204',
 '1203',
 '1202',
 '1201',
 '1116',
 '1115',
 '1114',
 '1113',
 '1112',
 '1016',
 '1015',
 '1014',
 '0912',
 '0911',
 '0910',
 '0909',
 '0908',
 '0907',
 '0906',
 '0905',
 '0904',
 '0903',
 '0902',
 '0901',
 '0805',
 '0804',
 '0803',
 '0913',
 '0914',
 '1013',
 '1012',
 '1011',
 '1010',
 '1009',
 '1008',
 '1007',
 '1006',
 '1005',
 '1004',
 '1003',
 '1002',
 '1001',
 '0916',
 '0915',
 '1306',
 '1307',
 '1624',
 '1623',
 '1622',
 '1621',
 '1620',
 '1619',
 '1618',
 '1617',
 '1616',
 '1615',
 '1614',
 '1613',
 '1612',
 '1611',
 '1610',
 '1625',
 '1701',
 '1815',
 '1814',
 '1813',
 '1812',
 '1811',
 '1810',
 '1809',
 '1808',
 '1807',
 '1806',
 '1805',
 '1804',
 '1803',
 '1802',
 '1801',
 '1609',
 '1608',
 '1607',
 '1322',
 '1321',
 '1320',
 '1319',
 '1318',
 '1317',
 

In [62]:
import requests
import json
import pandas as pd
from tqdm import tqdm
url = "https://www.hyundai.com/kr/ko/gw/customer-support/v1/purchase-guide/agencies"

params = {
    "latitude": 37.4646651,
    "longitude": 127.0430687,
    "mapLatitude": "",
    "mapLongitude": "",
    "radius": "",
    "pageSize": 10,
    "pageNo": 1,
    "searchFilter": "",
    "agencyTypeCode": "",
    "deliveryAreaCode": "01",
    "localAreaCode": "0101",
    "findType": "W",
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://www.hyundai.com/kr/ko/e/vehicles/purchase-guide",
    "Origin": "https://www.hyundai.com",
}

r = requests.get(url, params=params, headers=headers, timeout=20)
print(r.status_code)


rows = []
### 이제부터 for문을 돌아본다...
for sido in tqdm(sido_list):
    params['deliveryAreaCode']=sido
    for gu in tqdm(gugun_list):
        params['localAreaCode']=gu
        r = requests.get(url, params=params, headers=headers, timeout=20)
        (r.status_code)

        r = r.text.strip("'")

        # 1) JSON 파싱
        obj = json.loads(r)
        stores = obj["data"]["list"]


        # 2) "유의미한" 컬럼만 추출
        keep_cols = [
            "agencyCode", "agencyName", "agencyTypeCode",
            "agencyAddress", "agencyTel",
            "lattitude", "longitude",
            "distance", "displayCarCount", "carMasterCount"
        ]

        
        for s in stores:
            row = {k: s.get(k) for k in keep_cols}

            # 3) 타입 정리(숫자 변환)
            row["distance_km"] = float(row["distance"]) if row.get("distance") not in (None, "") else None
            row["lat"] = float(row["lattitude"]) if row.get("lattitude") not in (None, "") else None
            row["lon"] = float(row["longitude"]) if row.get("longitude") not in (None, "") else None
            row["displayCarCount"] = int(row["displayCarCount"]) if row.get("displayCarCount") not in (None, "") else None
            row["carMasterCount"] = int(row["carMasterCount"]) if row.get("carMasterCount") not in (None, "") else None

            
            row["is_showroom"] = (row["displayCarCount"] or 0) > 0

            type_map = {"1": "지점/전시장", "2": "대리점"}
            row["agencyType"] = type_map.get(str(row["agencyTypeCode"]), "기타")

            row["sido_code"] = sido
            row["gugun_code"] = gu

            print(row)
            rows.append(row)




200


  0%|          | 0/16 [00:00<?, ?it/s]

{'agencyCode': 'DB39004', 'agencyName': '도곡중앙', 'agencyTypeCode': '2', 'agencyAddress': '서울특별시 강남구 언주로 122 (도곡동) (06295)', 'agencyTel': '02-3463-5200', 'lattitude': '37.487082477134', 'longitude': '127.051017587398', 'distance': '2.6', 'displayCarCount': 0, 'carMasterCount': 18, 'distance_km': 2.6, 'lat': 37.487082477134, 'lon': 127.051017587398, 'is_showroom': False, 'agencyType': '대리점', 'sido_code': '01', 'gugun_code': '0101'}
{'agencyCode': 'B39', 'agencyName': '강남대로', 'agencyTypeCode': '1', 'agencyAddress': '서울특별시 강남구 강남대로 292 현대자동차(주) 강남대로지점  전시장 (06258)', 'agencyTel': '02-3461-1990', 'lattitude': '37.4893966742', 'longitude': '127.032205066703', 'distance': '2.9', 'displayCarCount': 8, 'carMasterCount': 36, 'distance_km': 2.9, 'lat': 37.4893966742, 'lon': 127.032205066703, 'is_showroom': True, 'agencyType': '지점/전시장', 'sido_code': '01', 'gugun_code': '0101'}
{'agencyCode': 'DB44007', 'agencyName': '언주', 'agencyTypeCode': '2', 'agencyAddress': '서울특별시 강남구 도곡로 251 1층 전시장 2층 사무실 (0623

{'agencyCode': 'C16', 'agencyName': '옥수', 'agencyTypeCode': '1', 'agencyAddress': '서울특별시 성동구 독서당로 203 공공복합청사 1층 현대자동차 (04732)', 'agencyTel': '02-2298-3500', 'lattitude': '37.543640267785', 'longitude': '127.013414924954', 'distance': '9.2', 'displayCarCount': 2, 'carMasterCount': 15, 'distance_km': 9.2, 'lat': 37.543640267785, 'lon': 127.013414924954, 'is_showroom': True, 'agencyType': '지점/전시장', 'sido_code': '01', 'gugun_code': '0116'}
{'agencyCode': 'DB22004', 'agencyName': '한양', 'agencyTypeCode': '2', 'agencyAddress': '서울특별시 성동구 왕십리로 223 (행당동, 동우빌딩) (04764)', 'agencyTel': '02-2299-5400', 'lattitude': '37.557034666231', 'longitude': '127.042458111531', 'distance': '10.3', 'displayCarCount': 0, 'carMasterCount': 20, 'distance_km': 10.3, 'lat': 37.557034666231, 'lon': 127.042458111531, 'is_showroom': False, 'agencyType': '대리점', 'sido_code': '01', 'gugun_code': '0116'}
{'agencyCode': 'B19', 'agencyName': '성동', 'agencyTypeCode': '1', 'agencyAddress': '서울특별시 성동구 천호대로 430 대동빌딩1층,2층 현대자동차 (0

{'agencyCode': 'DB39003', 'agencyName': '서울', 'agencyTypeCode': '2', 'agencyAddress': '서울특별시 서초구 매헌로 16 하이브랜드 1층 (06771)', 'agencyTel': '02-597-7682', 'lattitude': '37.462467348157', 'longitude': '127.036947909974', 'distance': '.6', 'displayCarCount': 2, 'carMasterCount': 19, 'distance_km': 0.6, 'lat': 37.462467348157, 'lon': 127.036947909974, 'is_showroom': True, 'agencyType': '대리점', 'sido_code': '01', 'gugun_code': '0115'}
{'agencyCode': 'DB39006', 'agencyName': '내곡', 'agencyTypeCode': '2', 'agencyAddress': '서울특별시 서초구 청계산로 217 (신원동) (06802)', 'agencyTel': '02-577-8775', 'lattitude': '37.446323431928', 'longitude': '127.057319631369', 'distance': '2.4', 'displayCarCount': 1, 'carMasterCount': 18, 'distance_km': 2.4, 'lat': 37.446323431928, 'lon': 127.057319631369, 'is_showroom': True, 'agencyType': '대리점', 'sido_code': '01', 'gugun_code': '0115'}
{'agencyCode': 'DB57004', 'agencyName': '신강남', 'agencyTypeCode': '2', 'agencyAddress': '서울특별시 서초구 사임당로 115 1층 (서초동) (06636)', 'agencyTel': '

...

{'agencyCode': 'DL11003', 'agencyName': '월산', 'agencyTypeCode': '2', 'agencyAddress': '광주광역시 남구 대남대로 357 (월산동) (61612)', 'agencyTel': '062-364-3700', 'lattitude': '35.143230330267', 'longitude': '126.891848057623', 'distance': '258.5', 'displayCarCount': 1, 'carMasterCount': 18, 'distance_km': 258.5, 'lat': 35.143230330267, 'lon': 126.891848057623, 'is_showroom': True, 'agencyType': '대리점', 'sido_code': '08', 'gugun_code': '0802'}
{'agencyCode': 'DL11005', 'agencyName': '진월', 'agencyTypeCode': '2', 'agencyAddress': '광주광역시 남구 서문대로 676 (진월동) (61710)', 'agencyTel': '062-654-7111', 'lattitude': '35.119661233549', 'longitude': '126.899411541375', 'distance': '261.1', 'displayCarCount': 4, 'carMasterCount': 17, 'distance_km': 261.1, 'lat': 35.119661233549, 'lon': 126.899411541375, 'is_showroom': True, 'agencyType': '대리점', 'sido_code': '08', 'gugun_code': '0802'}
{'agencyCode': 'DL17002', 'agencyName': '첨단', 'agencyTypeCode': '2', 'agencyAddress': '광주광역시 광산구 임방울대로 785 (쌍암동) (62274)', 'agencyTe

 94%|█████████▍| 15/16 [17:50<01:14, 74.48s/it]

{'agencyCode': 'DT12003', 'agencyName': '제주표선', 'agencyTypeCode': '2', 'agencyAddress': '제주특별자치도 서귀포시 표선면 일주동로 6464 . (63626)', 'agencyTel': '064-787-1530', 'lattitude': '33.307682249121', 'longitude': '126.781226809368', 'distance': '462.8', 'displayCarCount': 5, 'carMasterCount': 6, 'distance_km': 462.8, 'lat': 33.307682249121, 'lon': 126.781226809368, 'is_showroom': True, 'agencyType': '대리점', 'sido_code': '14', 'gugun_code': '1401'}
{'agencyCode': 'DT13002', 'agencyName': '제주신화', 'agencyTypeCode': '2', 'agencyAddress': '제주특별자치도 서귀포시 안덕면 신화역사로 434 1층 전시장, 2층 사무실 (63523)', 'agencyTel': '064-792-3301', 'lattitude': '33.309575842777', 'longitude': '126.333882880174', 'distance': '466.5', 'displayCarCount': 2, 'carMasterCount': 7, 'distance_km': 466.5, 'lat': 33.309575842777, 'lon': 126.333882880174, 'is_showroom': True, 'agencyType': '대리점', 'sido_code': '14', 'gugun_code': '1401'}
{'agencyCode': 'T13', 'agencyName': '서귀포', 'agencyTypeCode': '1', 'agencyAddress': '제주특별자치도 서귀포시 일주동로 8914 

{'agencyCode': 'DT14004', 'agencyName': '제주북촌', 'agencyTypeCode': '2', 'agencyAddress': '제주특별자치도 제주시 조천읍 북촌14길 2  (63339)', 'agencyTel': '064-783-2323', 'lattitude': '33.549597755723', 'longitude': '126.698147032969', 'distance': '436.5', 'displayCarCount': 6, 'carMasterCount': 6, 'distance_km': 436.5, 'lat': 33.549597755723, 'lon': 126.698147032969, 'is_showroom': True, 'agencyType': '대리점', 'sido_code': '14', 'gugun_code': '1402'}
{'agencyCode': 'T12', 'agencyName': '제주번영로', 'agencyTypeCode': '1', 'agencyAddress': '제주특별자치도 제주시 연북로 875 현대자동차(주)제주번영로지점 (63304)', 'agencyTel': '064-722-8001', 'lattitude': '33.503612418668', 'longitude': '126.566349664057', 'distance': '442.6', 'displayCarCount': 15, 'carMasterCount': 15, 'distance_km': 442.6, 'lat': 33.503612418668, 'lon': 126.566349664057, 'is_showroom': True, 'agencyType': '지점/전시장', 'sido_code': '14', 'gugun_code': '1402'}
{'agencyCode': 'DT14001', 'agencyName': '일도', 'agencyTypeCode': '2', 'agencyAddress': '제주특별자치도 제주시 연삼로 526 (일도이동) (

100%|██████████| 16/16 [19:11<00:00, 71.99s/it]


In [63]:
df = pd.DataFrame(rows)

In [64]:
df.to_pickle("./hyundai_store.pkl")

In [67]:
df['sido_code'].unique()

array(['01', '06', '07', '02', '05', '03', '04', '15', '16', '11', '13',
       '12', '18', '10', '08', '14'], dtype=object)